Read data from csv

In [ ]:
from pandas import read_csv
filename = 'dataset//earnings_split/EAR_4MMN_CUR_NB_A-filtered-2021-03-21.csv'
df_incomes_columns_csv = ["ref_area.label","classif1.label","time","obs_value"]
df_incomes_columns_names = ["Country","Currency","Year","Income"]
df_incomes = read_csv(filename,usecols=df_incomes_columns_csv)
df_incomes = df_incomes[(df_incomes["classif1.label"]=="Currency: U.S. dollars") & (df_incomes["time"]>2009)].drop("classif1.label", axis=1)

df_incomes = df_incomes.rename(columns={k:v for k,v in zip(df_incomes_columns_csv,df_incomes_columns_names)})
df_incomes.head()

In [ ]:
from pandas import merge as pd_merge
filename = 'dataset//earnings_split/EAR_4MTH_SEX_ECO_CUR_NB_A-filtered-2021-03-21.csv'
df_field_incomes_columns_csv = ["ref_area.label","classif1.label","classif2.label","time","obs_value"]
df_field_incomes_columns_names = ["Country","Field","Currency","Year","Income"]
df_field_incomes = read_csv(filename,usecols=df_field_incomes_columns_csv)
df_field_incomes = df_field_incomes[(df_field_incomes["classif2.label"]=="Currency: U.S. dollars") & (df_field_incomes["time"]>2009)].drop("classif2.label", axis=1)

df_field_incomes = df_field_incomes.rename(columns={k:v for k,v in zip(df_field_incomes_columns_csv,df_field_incomes_columns_names)})
df_avg_income = df_field_incomes[df_field_incomes["Field"]=="Economic activity (Aggregate): Total"].drop("Field", axis=1).rename(columns={"Income":"Average Income"})
df_field_incomes["Field"] = list(map(lambda text: text.split(":")[1][1:],df_field_incomes["Field"]))
df_field_incomes = pd_merge(df_field_incomes,df_avg_income, on=["Year", "Country"],how="inner")
df_field_incomes.head()

In [ ]:
filename = 'dataset//living_index//living_index.csv'
indices_columns = ["Country","Cost of Living Index","Year", "Local Purchasing Power Index"]
df_indices = read_csv(filename,usecols=indices_columns)
indices_by_country = [dfCountry  for country,dfCountry in df_indices.groupby('Country')]

In [ ]:
from math import ceil
import matplotlib.pyplot as plt
def draw_df_by_groups(dfs,x_name,y_name,legend,groupSize=5):
    plots_count = len(dfs)
    plot_groups = int(ceil(plots_count/groupSize))
    fig, axs = plt.subplots(plot_groups,figsize=(18, plot_groups*5))
    for i in range(plots_count):
        dfs[i].plot(x=x_name,y=y_name,ax=axs[i//groupSize])
        if i%groupSize == groupSize-1 or i==plots_count-1:
            axs[i//groupSize].legend(legend[i//groupSize*groupSize:i+1])
    fig.show()

Drawing commented out because it takes too much space

In [ ]:
# countries = [country["Country"].values[0] for country in indices_by_country]      
# draw_df_by_groups(indices_by_country,"Year","Cost of Living Index",countries,5)   

## Purchasing power index

In [ ]:
df_indices_income_favg = pd_merge(df_indices,df_field_incomes, on=["Year", "Country"],how="inner")
print("Countries count after merge",df_indices_income_favg["Country"].unique().size)
df_indices_income_favg["Field Purchasing Power Index"] = df_indices_income_favg["Local Purchasing Power Index"]/df_indices_income_favg["Average Income"]*df_indices_income_favg["Income"]
df_indices_income_favg.head()

In [ ]:
# temporary fix for Poland
# df_indices_income_favg["Income"] =((df_indices_income_favg["Year"]>2017) & (df_indices_income_favg["Country"]=="Poland"))*9*df_indices_income_favg["Income"]+df_indices_income_favg["Income"]

In [ ]:
# Polska/Niemcy/Usa
def plot_groupby(df_groupedby,x_name,y_name,title=""):
    fig, ax = plt.subplots(figsize=(18,18))

    for name, group in df_groupedby:
        group.plot(x=x_name,y=y_name, ax=ax, label=name,title=title)      #kind='kde',
    fig.show()
countries = ["Poland","Germany","United States"]

for country in countries:
    plot_groupby(df_indices_income_favg[df_indices_income_favg['Country'] == country].groupby("Field"),x_name="Year",y_name="Field Purchasing Power Index",title=country)


## Fourier tranform

In [ ]:
poland_Cost_of_living = df_indices.loc[df_indices['Country'] == "Poland"][["Cost of Living Index","Year"]]

In [ ]:
years=poland_Cost_of_living.values[:,1]
cost_of_living_index=poland_Cost_of_living.values[:,0]
plt.plot(years,cost_of_living_index,'o')

In [ ]:
from scipy.fft import fft, fftfreq
import numpy as np
# Number of sample points
N = cost_of_living_index.size
# sample spacing
T = 1
yf = fft(cost_of_living_index)
xf = fftfreq(N, T)[:N//2]
import matplotlib.pyplot as plt
plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
plt.grid()
plt.show()

## Linear regression

In [ ]:
x=poland_Cost_of_living.values[:,1]
y=poland_Cost_of_living.values[:,0]
predict_x = [x for x in range(2009,2026)]

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression(n_jobs = -1)
model.fit(np.reshape(x,(-1,1)), y)
predicted_y = model.predict(np.reshape(predict_x,(-1,1)))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
plt.scatter(x, y, color = "red")
plt.plot(predict_x, predicted_y, color = "green")
plt.title("Cost of living index in Poland")
plt.xlabel("Year")
plt.ylabel("IndexLevel")
plt.show()

In [ ]:
#ToDo regresja wielomianowa

# Cost of living - compare countries

In [ ]:
# Get first year of a series
first_year = list(results_by_year.keys())[0]

# Get country names for that year
first_year_countries = results_by_year[first_year]['City']

# Use mask to select countires in the first year that are present in every series
first_year_mask = [True for i in range(len(first_year_countries))]
for year in list(results_by_year.keys())[1:]:
    curr_year_countries = results_by_year[year]['City']
    mask = np.in1d(first_year_countries, curr_year_countries)
    first_year_mask &= mask

# Get list of countires that are present in every series
list_of_countries = list(first_year_countries[first_year_mask])

In [ ]:
# Container for standardized series
filtered_countries = []

# Filter countries in series
for year in list(results_by_year.keys()):
    # selecting rows based on condition 
    curr_series = results_by_year[year]
    filtered_curr_series = curr_series[curr_series['City'].isin(list_of_countries)] 
    filtered_countries.append(filtered_curr_series)

In [ ]:
# Get series for each country
country_series = {}
for country in list_of_countries:
    country_series[country] = []
    
# Fill country series
for year_dataframe in filtered_countries:
    for row in year_dataframe.itertuples():
        country_series[row[1]].append(row[2])

## Linear comparison

In [ ]:
from pandas import Series

# Create and fill a matrix for linear distance between countries
heatmap_linear = np.zeros((65, 65))

for i in range(len(list_of_countries)):
    for j in range(len(list_of_countries)):
        dist = (np.linalg.norm(Series(country_series[list_of_countries[i]])-Series(country_series[list_of_countries[j]])))
        heatmap_linear[i][j] = dist

In [ ]:
plt.figure(figsize=(20, 20))
plt.xticks(ticks=np.arange(len(list_of_countries)),labels=list_of_countries,rotation=90)
plt.yticks(ticks=np.arange(len(list_of_countries)),labels=list_of_countries)
plt.imshow(heatmap_linear, cmap='cool',interpolation="nearest")

## Differential comparison

In [ ]:
# Differential values
diff_country_series = {}

for key, value in country_series.items():
    diff_country_series[key] = list(Series(value).diff(1))[1:]

In [ ]:
# Create and fill a matrix for linear distance between countries
heatmap_differential = np.zeros((65, 65))

for i in range(len(list_of_countries)):
    for j in range(len(list_of_countries)):
        dist = (np.linalg.norm(Series(diff_country_series[list_of_countries[i]])-Series(diff_country_series[list_of_countries[j]])))
        heatmap_differential[i][j] = dist

In [ ]:
plt.figure(figsize=(20, 20))
plt.xticks(ticks=np.arange(len(list_of_countries)),labels=list_of_countries,rotation=90)
plt.yticks(ticks=np.arange(len(list_of_countries)),labels=list_of_countries)
plt.imshow(heatmap_differential, cmap='cool',interpolation="nearest")